In [1]:
# For tips on running notebooks in Google Colab, see
# https://pytorch.org/tutorials/beginner/colab
%matplotlib inline

In [4]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
#from torchvision import datasets, transforms

from typing import Tuple, List
from collections import deque

import torch.optim as optim
from torch.optim import lr_scheduler
from torch.optim import Optimizer

import numpy as np
from torch import Tensor

import torch.nn as nn
import torch.nn.functional as F
from torch.nn.modules.loss import _Loss

In [46]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")
torch.set_default_device(device)

Using cuda device


In [47]:
torch.manual_seed(20190325);

In [48]:
import pandas as pd
import numpy as np

data_url = "http://lib.stat.cmu.edu/datasets/boston"
raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
data = np.hstack([raw_df.values[::2, :], raw_df.values[1::2, :2]])
target = raw_df.values[1::2, 2]

In [49]:
from sklearn.preprocessing import StandardScaler
s = StandardScaler()
data = s.fit_transform(data)

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.3, random_state=80718)

y_train, y_test = y_train.reshape(-1, 1), y_test.reshape(-1, 1)

X_train, X_test, y_train, y_test = Tensor(X_train), Tensor(X_test), Tensor(y_train), Tensor(y_test)
X_train = X_train.to(device)
X_test = X_test.to(device)
y_train = y_train.to(device)
y_test = y_test.to(device)

In [50]:
class HousePricesModel(nn.Module):
  def __init__(self, hidden_size: int = 13):
    super().__init__()
    self.fc1 = nn.Linear(13, hidden_size)
    self.fc2 = nn.Linear(hidden_size, 1)

  def forward(self, x: Tensor) -> Tensor:
    #assert_dim(x, 2)
    assert x.shape[1] == 13
    x = self.fc1(x)
    x = torch.sigmoid(x)
    return self.fc2(x)

In [51]:
model = HousePricesModel().to(device)
print(model)

HousePricesModel(
  (fc1): Linear(in_features=13, out_features=13, bias=True)
  (fc2): Linear(in_features=13, out_features=1, bias=True)
)


In [52]:
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

In [53]:
criterion = nn.MSELoss()

In [54]:
print(X_train[0])
print(y_train[0])


tensor([-0.4125, -0.4877, -1.3069, -0.2726, -0.8353,  1.2286, -0.5112,  1.0777,
        -0.7529, -1.1061,  0.1130,  0.4411, -1.0265], device='cuda:0')
tensor([36.2000], device='cuda:0')


In [55]:
tns = X_train[0].unsqueeze(0)
#tns = tns.to(device)
print(tns)
print(tns.device)
print(model.forward(tns))

tensor([[-0.4125, -0.4877, -1.3069, -0.2726, -0.8353,  1.2286, -0.5112,  1.0777,
         -0.7529, -1.1061,  0.1130,  0.4411, -1.0265]], device='cuda:0')
cuda:0
tensor([[0.1196]], device='cuda:0', grad_fn=<AddmmBackward0>)


In [58]:
class PyTorchTrainer(object):

  def __init__(self, model: nn.Module, optim: Optimizer, criterion: _Loss):
    self.model = model
    self.optim = optim
    self.loss = criterion
    self._check_optim_net_aligned()

  def _check_optim_net_aligned(self):
    assert self.optim.param_groups[0]['params'] == list(self.model.parameters())

  def _generate_batches(self, X: Tensor, y: Tensor, size: int = 32) -> Tuple[Tensor]:
    N = X.shape[0]
    for ii in range(0, N, size):
      X_batch, y_batch = X[ii:ii+size], y[ii:ii+size]
      yield X_batch, y_batch

  def fit(self, X_train: Tensor, y_train: Tensor, X_test: Tensor, y_test: Tensor,epochs: int=100,eval_every: int=10, batch_size: int=32):
    for e in range(epochs):
      #X_train, y_train = permute_data(X_train, y_train)
      batch_generator = self._generate_batches(X_train, y_train, batch_size)

      for ii, (X_batch, y_batch) in enumerate(batch_generator):
        #print(X_batch.device)
        self.optim.zero_grad()
        output = self.model(X_batch)
        loss = self.loss(output, y_batch)
        loss.backward()
        self.optim.step()
      output = self.model(X_test)
      loss = self.loss(output, y_test)
      print(e, loss)

In [67]:
net = HousePricesModel()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)
criterion = nn.MSELoss()

trainer = PyTorchTrainer(net, optimizer, criterion)
#print(y_test.device)
trainer.fit(X_train, y_train, X_test, y_test, epochs=100, eval_every=1)

0 tensor(221.4138, device='cuda:0', grad_fn=<MseLossBackward0>)
1 tensor(51.2004, device='cuda:0', grad_fn=<MseLossBackward0>)
2 tensor(45.5998, device='cuda:0', grad_fn=<MseLossBackward0>)
3 tensor(34.0103, device='cuda:0', grad_fn=<MseLossBackward0>)
4 tensor(31.1356, device='cuda:0', grad_fn=<MseLossBackward0>)
5 tensor(30.1122, device='cuda:0', grad_fn=<MseLossBackward0>)
6 tensor(30.2407, device='cuda:0', grad_fn=<MseLossBackward0>)
7 tensor(28.9635, device='cuda:0', grad_fn=<MseLossBackward0>)
8 tensor(28.0983, device='cuda:0', grad_fn=<MseLossBackward0>)
9 tensor(27.5829, device='cuda:0', grad_fn=<MseLossBackward0>)
10 tensor(27.0288, device='cuda:0', grad_fn=<MseLossBackward0>)
11 tensor(26.5478, device='cuda:0', grad_fn=<MseLossBackward0>)
12 tensor(26.1007, device='cuda:0', grad_fn=<MseLossBackward0>)
13 tensor(25.6753, device='cuda:0', grad_fn=<MseLossBackward0>)
14 tensor(25.2903, device='cuda:0', grad_fn=<MseLossBackward0>)
15 tensor(24.9228, device='cuda:0', grad_fn=<MseL

In [61]:
print(model.forward(tns))

tensor([[0.1196]], device='cuda:0', grad_fn=<AddmmBackward0>)


In [62]:
model(tns)

tensor([[0.1196]], device='cuda:0', grad_fn=<AddmmBackward0>)

In [70]:
print(X_train[0])
print(y_train[0])

tensor([-0.4125, -0.4877, -1.3069, -0.2726, -0.8353,  1.2286, -0.5112,  1.0777,
        -0.7529, -1.1061,  0.1130,  0.4411, -1.0265], device='cuda:0')
tensor([36.2000], device='cuda:0')


In [71]:
net(X_train[0].unsqueeze(0))

tensor([[30.3028]], device='cuda:0', grad_fn=<AddmmBackward0>)